## Extract Alternative and Previous Names

In [1]:
import pandas as pd
import numpy as np
import time
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')

t0=time.time()

df=pd.read_csv("../csvResults/dfMerged_50000_65752.csv").iloc[:,[0,3,34,43,47]]

df['data']=df['data'].str.lower()

t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

display(df.info(null_counts=True),df.head())

Total time is 0.013879 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20969 entries, 0 to 20968
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            20969 non-null  int64 
 1   name          13602 non-null  object
 2   data          8680 non-null   object
 3   address_city  9320 non-null   object
 4   agent_city    3044 non-null   object
dtypes: int64(1), object(4)
memory usage: 819.2+ KB


None

,ID,name,data,address_city,agent_city
0,50000,NLB Water LLC,"{'most_recent': [{'datum': {'id': 87522262, 't...",NaN,NaN
1,50001,Glucan Biorenewables LLC,NaN,Saint Louis,NaN
2,50002,NaN,NaN,NaN,NaN
3,50003,ETHOS INTERNATIONAL CORPORATION,"{'most_recent': [{'datum': {'id': 38743134, 't...",SALEM,NaN
4,50004,NaN,NaN,NaN,NaN


In [2]:
t0=time.time()

df1=pd.read_csv("../csvResults/dfMerged_0_49999.csv").iloc[:,[0,6,37,46,50]]

df1['data']=df1['data'].str.lower()

t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

display(df1.info(null_counts=True),df1.head())

Total time is 0.030244 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41980 entries, 0 to 41979
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            41980 non-null  int64 
 1   name          35812 non-null  object
 2   data          22778 non-null  object
 3   address_city  23781 non-null  object
 4   agent_city    8257 non-null   object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


None

,ID,name,data,address_city,agent_city
0,0,"Close In Solutions, LLC","{'most_recent': [{'datum': {'id': 40213152, 't...",ROUND ROCK,NaN
1,3,Valley Business Solutions LLC,"{'most_recent': [{'datum': {'id': 26997008, 't...",NaN,NaN
2,3,Valley Payroll Management and Business Solutio...,"{'most_recent': [{'datum': {'id': 27009689, 't...",NaN,NaN
3,4,"RAILIAS HOLDINGS, LLC","{'most_recent': [{'datum': {'id': 118648080, '...",SAN DIEGO,SAN DIEGO
4,6,FAIRDALE ORTHODONTIC COMPANY,NaN,NaN,NaN


In [3]:
combine=pd.concat([df1,df],axis=0).sort_values(by=['ID']).reset_index(drop=True)

display(combine.info(),combine.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62949 entries, 0 to 62948
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            62949 non-null  int64 
 1   name          49414 non-null  object
 2   data          31458 non-null  object
 3   address_city  33101 non-null  object
 4   agent_city    11301 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


None

,ID,name,data,address_city,agent_city
0,0,"Close In Solutions, LLC","{'most_recent': [{'datum': {'id': 40213152, 't...",ROUND ROCK,NaN
1,3,Valley Business Solutions LLC,"{'most_recent': [{'datum': {'id': 26997008, 't...",NaN,NaN
2,3,Valley Payroll Management and Business Solutio...,"{'most_recent': [{'datum': {'id': 27009689, 't...",NaN,NaN
3,4,"RAILIAS HOLDINGS, LLC","{'most_recent': [{'datum': {'id': 118648080, '...",SAN DIEGO,SAN DIEGO
4,6,FAIRDALE ORTHODONTIC COMPANY,NaN,NaN,NaN


In [4]:
combine['data']=combine['data'].str.replace("'% '",'')
combine['data']=combine['data'].str.replace(r"\\n"," ")
combine1=combine

display(combine1.info(null_counts=True),combine1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62949 entries, 0 to 62948
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            62949 non-null  int64 
 1   name          49414 non-null  object
 2   data          31458 non-null  object
 3   address_city  33101 non-null  object
 4   agent_city    11301 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


None

,ID,name,data,address_city,agent_city
0,0,"Close In Solutions, LLC","{'most_recent': [{'datum': {'id': 40213152, 't...",ROUND ROCK,NaN
1,3,Valley Business Solutions LLC,"{'most_recent': [{'datum': {'id': 26997008, 't...",NaN,NaN
2,3,Valley Payroll Management and Business Solutio...,"{'most_recent': [{'datum': {'id': 27009689, 't...",NaN,NaN
3,4,"RAILIAS HOLDINGS, LLC","{'most_recent': [{'datum': {'id': 118648080, '...",SAN DIEGO,SAN DIEGO
4,6,FAIRDALE ORTHODONTIC COMPANY,NaN,NaN,NaN


In [5]:
### consecutive regex processes were performed to extract the organization name from the
### alternative_names and previous_names columns. When a match is absent, the if statement
### skips that record. If a match is found, the first regex variable is used to find all
### instances that match the pattern, then the second regex variable is utilized to further
### refine the string to reduce it to the organization name. A list is created using the
### nested for loop for each record. If multiple names are present, a nested list is created
### for that record. After the for loops are finished running, the final list is created and
### printed for reviewing. The description applies for processing both the alternative_names
### and previous_names columns.

regex="'companyaddress', 'description': '[a-z0-9].+?'"
regex1="'[a-z].+': "

b=len(combine1)
sub_finalAlt = []
finalAlt = []

for j in range(b):
    
    try:
        if pd.isna(combine1['data'][j]) is True:
            sub_finalAlt.append(np.nan)

        else:
            a=combine1['data'][j]
            match=re.findall(regex,a)
            c=len(match)
            
            if c == 0:
#                 print(re.sub(regex1,"",match))
#                 match1=re.sub(regex1,"",match)
                sub_finalAlt.append(np.nan)

            elif c == 1:
#                 print(re.sub(regex1,"",match[0]))
                match1=re.sub(regex1,"",match[0])
                sub_finalAlt.append(match1)    
    
            elif c > 1:
#                 print(re.sub(regex1,"",match[1]))
                match1=re.sub(regex1,"",match[1])
                sub_finalAlt.append(match1)

            

    except:
        print('this is bad')


    finalAlt.append(sub_finalAlt)
    sub_finalAlt = []

print(finalAlt)

[["'23 meandering way, round rock, tx, 78664'"], ["'huntsville, al'"], ["'alexandria, al'"], ["'10251 vista sorrento pkwy ste 300, san diego, ca, 92121'"], [nan], [nan], ["'10623 winchelsea dr, austin, tx, 78750'"], ["'10275 science center drive, san diego, ca, 92121'"], ["'6081 meridian avenue, suite 70, box 182, san jose, ca, 95120'"], ["'642 via umbroso, san clemente, ca, 92672'"], ["'arjang & co inc., 38 north road, great neck, ny, 11024'"], ["'10680 treena street, suite 600, san diego, ca, 92131'"], ["'2 musick, irvine, ca, 92618'"], [nan], [nan], [nan], [nan], ["'25811 lawson st, black diamond, wa, 98010-9795, united states'"], ["'5412 beebe st. ne, albuquerque, nm, 87111, usa'"], ["'211 nw 5th avenue, hallandale, fl, 33009'"], [nan], ["'6005 bluffton road, fort wayne, in, 46809'"], ["'po box 131526, ann arbor, mi, 48113'"], ["'1642 ross lane, ashland, or, 97520, united states of america'"], [nan], [nan], ["'55 e 3rd ave san mateo ca 94401'"], ["'4450 enterprise street #101 suite

In [6]:
test=pd.DataFrame(finalAlt,columns=['addresses'])
test['addresses']=test['addresses'].str.replace("'","")

display(test)

,addresses
0,"23 meandering way, round rock, tx, 78664"
1,"huntsville, al"
2,"alexandria, al"
3,"10251 vista sorrento pkwy ste 300, san diego, ..."
4,NaN
...,...
62944,NaN
62945,"500 commerce st ste 700, fort worth, tx, 76102"
62946,"3609 marquis dr, garland, tx, 75042"
62947,NaN


In [35]:
patternCities = '|'.join(['fort worth','round rock'])

In [30]:
# importing pandas as pd
import pandas as pd

# importing re for regular expressions
import re

# Creating the Series
sr = pd.Series(test['addresses'])

# Creating the index
# idx = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']

# set the index
# sr.index = idx

# Print the series
print(sr)


0                 23 meandering way, round rock, tx, 78664
1                                           huntsville, al
2                                           alexandria, al
3        10251 vista sorrento pkwy ste 300, san diego, ...
4                                                      NaN
                               ...                        
62944                                                  NaN
62945       500 commerce st ste 700, fort worth, tx, 76102
62946                  3609 marquis dr, garland, tx, 75042
62947                                                  NaN
62948                      438 tyler rd, merced, ca, 95341
Name: addresses, Length: 62949, dtype: object


In [82]:
# result = sr.str.extract(pat = '(patternCities)')
patternCities = '|'.join(['fort worth','round rock'])
cityListSub=[]
cityListFinal=[]

for tt in range(len(test)):

    try:
        aa=test['addresses'][tt]
        match33=re.findall(patternCities,aa)
        cityListSub.append(match33[tt])
        
    except:
        cityListSub.append(np.nan)
    
#     cityListFinal.append(cityListSub)
#     cityListSub = []
    
cityListSub

['round rock',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 

In [83]:
cityListSub[0]

'round rock'

In [7]:
splitTest=test['addresses'].str.split(',',expand=True)

display(splitTest)

,0,1,2,3,4,5,6,7,8,9,10
0,23 meandering way,round rock,tx,78664,None,None,None,None,None,None,None
1,huntsville,al,None,None,None,None,None,None,None,None,None
2,alexandria,al,None,None,None,None,None,None,None,None,None
3,10251 vista sorrento pkwy ste 300,san diego,ca,92121,None,None,None,None,None,None,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
62944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62945,500 commerce st ste 700,fort worth,tx,76102,None,None,None,None,None,None,None
62946,3609 marquis dr,garland,tx,75042,None,None,None,None,None,None,None
62947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
splitTest.drop(labels=[7,8],axis=1,inplace=True)
splitTest=splitTest.applymap(lambda x: x.title() if pd.notnull(x) else '')

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,23 Meandering Way,Round Rock,Tx,78664,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,10251 Vista Sorrento Pkwy Ste 300,San Diego,Ca,92121,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,500 Commerce St Ste 700,Fort Worth,Tx,76102,,,,,
62946,3609 Marquis Dr,Garland,Tx,75042,,,,,
62947,,,,,,,,,


In [9]:
splitTest[6]=splitTest[6].str.lstrip()
splitTest[6]=splitTest[6].str.rstrip()

splitTest[5]=splitTest[5].str.lstrip()
splitTest[5]=splitTest[5].str.rstrip()

splitTest[4]=splitTest[4].str.lstrip()
splitTest[4]=splitTest[4].str.rstrip()

splitTest[3]=splitTest[3].str.lstrip()
splitTest[3]=splitTest[3].str.rstrip()

splitTest[2]=splitTest[2].str.lstrip()
splitTest[2]=splitTest[2].str.rstrip()

splitTest[1]=splitTest[1].str.lstrip()
splitTest[1]=splitTest[1].str.rstrip()

splitTest[0]=splitTest[0].str.lstrip()
splitTest[0]=splitTest[0].str.rstrip()


splitTest[6]=splitTest[6].str.replace("[0-9]| [0-9] ","")
splitTest[5]=splitTest[5].str.replace("[0-9]| [0-9] ","")
splitTest[4]=splitTest[4].str.replace("[0-9]| [0-9] ","")
splitTest[3]=splitTest[3].str.replace("[0-9]| [0-9] ","")
splitTest[2]=splitTest[2].str.replace("[0-9]| [0-9] ","")
splitTest[1]=splitTest[1].str.replace("[0-9]| [0-9] ","")
splitTest[0]=splitTest[0].str.replace("[0-9]| [0-9] ","")

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,Meandering Way,Round Rock,Tx,,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,Vista Sorrento Pkwy Ste,San Diego,Ca,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,Commerce St Ste,Fort Worth,Tx,,,,,,
62946,Marquis Dr,Garland,Tx,,,,,,
62947,,,,,,,,,


In [10]:
splitTest[6]=splitTest[6].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[5]=splitTest[5].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[4]=splitTest[4].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[3]=splitTest[3].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[2]=splitTest[2].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[1]=splitTest[1].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")
splitTest[0]=splitTest[0].str.replace("Usa|Us|United States|United States Of America|Aus|Of America|-","")

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,Meandering Way,Round Rock,Tx,,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,Vista Sorrento Pkwy Ste,San Diego,Ca,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,Commerce St Ste,Fort Worth,Tx,,,,,,
62946,Marquis Dr,Garland,Tx,,,,,,
62947,,,,,,,,,


In [11]:
splitTest[4]=splitTest[4].str.replace("LKB|TK L|Th Floor|Suite|^#|^#a|tria|Water St","")

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,Meandering Way,Round Rock,Tx,,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,Vista Sorrento Pkwy Ste,San Diego,Ca,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,Commerce St Ste,Fort Worth,Tx,,,,,,
62946,Marquis Dr,Garland,Tx,,,,,,
62947,,,,,,,,,


In [12]:
splitTest[3]=splitTest[3].str.replace("LKB|TK L|#A|#|\.|Ab|BA","")
splitTest[3]=splitTest[3].str.replace("LKB|TK L|Th Floor|Suite|^#|^#a|tria|Water St","")

splitTest[3]=splitTest[3].str.replace("Suite|Market Street|Thayer Ave|Ste|Commerce Drive|Allen St|United Penn Bank Bldg","")
splitTest[3]=splitTest[3].str.replace("Windstream Dr|Ste N|Williamsbridge Rd|Charwood Rd|Inc|Inc\.|Ttin|Bestgate Rd","")
splitTest[3]=splitTest[3].str.replace("Suite S|Ste\.|Th Floor|KK J|Telegraph Ave\.|Te|LW K|Attn: .*|Th Ave|Third Floor","")
splitTest[3]=splitTest[3].str.replace("MK E|Th Fl|Cranberry Twp|Centurion Pwky\. North|Unit|Section A|S Penataquit Avenue","")
splitTest[3]=splitTest[3].str.replace("Loct Grove Rd|BB L|MW E|Aec Suite|St\. Charles Ave\.|Light St\.|HT X|Rye|VGL","")
splitTest[3]=splitTest[3].str.replace("Po Box|wcb as|P\.O\. Box|Water St|P O Box|WcB As|Water St|Swenson Dr|Locust Grove Rd","")
splitTest[3]=splitTest[3].str.replace("Elk Grove Village|East Melrose Street|Cherry Lane Ct|Legraph Ave\.|N Bethesda","")
splitTest[3]=splitTest[3].str.replace("N Tonawanda|Apple Green Lane|Falls Rd|Aec|North Buckout St|Main Street","")
splitTest[3]=splitTest[3].str.replace("Centurion Pwky North|St Charles Ave|Light St |tin Tx|PO Box| legraph Ave","")

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,Meandering Way,Round Rock,Tx,,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,Vista Sorrento Pkwy Ste,San Diego,Ca,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,Commerce St Ste,Fort Worth,Tx,,,,,,
62946,Marquis Dr,Garland,Tx,,,,,,
62947,,,,,,,,,


In [13]:
splitTest[2]=splitTest[2].str.replace("LKB|TK L|Th Floor|Suite|^#|^#a|tria|Water St","")
splitTest[2]=splitTest[2].str.replace("#[A-Z]|#|\.|Ab$|BA|Jay.+?|Broadway.+?|.*.Lane|Temple Ave|Ste [A-Z]|Nd Ave So","")
splitTest[2]=splitTest[2].str.replace("Unit.+?|Po Box|Ste|Park Center Court|Hicksville Rd| Great Seveca Hwy|Montrose Road S","")
splitTest[2]=splitTest[2].str.replace(".*.Drive|International Circle|One Express Way|.*.Floor|.*.Ave|Sweet Home Rd","")
splitTest[2]=splitTest[2].str.replace("Allen St|.*.Street|.*.Road|Westmore Rd|Apt|Beltline Rd|Brookwood Farm Rd","")
splitTest[2]=splitTest[2].str.replace("Scott.*.|Spires Way|.*.Parkway|O Greenwch|One American Center|Nw|La Riviere Dr","")
splitTest[2]=splitTest[2].str.replace("Monroe St|Taxter Rd|.*.Center Court|.*.International|Coleman Road|Maxess Rd","")
splitTest[2]=splitTest[2].str.replace("Canopy Building.*|Avenue Of The.*.|.*.Avenue|Hwy East|Seneca St","")
splitTest[2]=splitTest[2].str.replace("Nd St|.*.Court|.*.Lane|Teal Dr|.*.Blvd|South Tower|Apo \(\)|Address|Sutton Pl S","")
splitTest[2]=splitTest[2].str.replace("P O Box|.*.Way|.*.Road|Mill Rd|Campus Dr|Th St E|Coconut Grove","")
splitTest[2]=splitTest[2].str.replace("Unit [A-Z]|.*.Expressway|.*.Boulevard|Scenic Gulf Dr.*|Historic Cobb Corner","")
splitTest[2]=splitTest[2].str.replace("Dutton Dr|Fountain Valley Dr|West Chester Pike|Building [A-Z]","")
splitTest[2]=splitTest[2].str.replace("Guilford Rd|Glynita Circle|C\/O.*|Hillmeade Rd|Winter Quarters Dr|Broad St|.*.Court","")
splitTest[2]=splitTest[2].str.replace("Hight Point Dr|tin Tx|Thousand Oaks|Brush Hollow Rd|N Druid Hills Rd","")
splitTest[2]=splitTest[2].str.replace("Uph|Building|Corporate Tax.*.|Highway [A-Z]|Samuel Morse Dr|Gudelsky Dr|L C Kohl","")
splitTest[2]=splitTest[2].str.replace("Vanessa St|Utopia Pkwy|Piccard Dr|.*.Bldg|Esq.*.|Medical Center Dr|W Christina Place","")
splitTest[2]=splitTest[2].str.replace("PO Box|Iii|ThirtyOne|W Th St|Th Fl|Apt [A-Z]|Indusl Dr|Rd\/|tin|Mylander Ln","")
splitTest[2]=splitTest[2].str.replace("N Haven St|Broadhallow Rd|Tax Department|W Ore St|\/|Llc|nue Of The Stars","")
splitTest[2]=splitTest[2].str.replace("^nue|Bryant St|ntura|Highway N|Airport Executive Park|rling Heights|Ns","")
splitTest[2]=splitTest[2].str.replace("nue Of The Americas| P O Box|nue \/|  Broadhollow Rd|Marigold Dr|Of The Americas|Xx","")
splitTest[2]=splitTest[2].str.replace("Highway.+[A-Z]|Et Al|Hwy  East|Dpt |Ave$|Dan Wykoff| Highway  N|Ben Clark| Phair St","")
splitTest[2]=splitTest[2].str.replace("Esq$|Bldg $|P O  Box | Penn Plaza| Broadhollow Rd|^os$| Route $","")

display(splitTest)

,0,1,2,3,4,5,6,9,10
0,Meandering Way,Round Rock,Tx,,,,,,
1,Huntsville,Al,,,,,,,
2,Alexandria,Al,,,,,,,
3,Vista Sorrento Pkwy Ste,San Diego,Ca,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,
62945,Commerce St Ste,Fort Worth,Tx,,,,,,
62946,Marquis Dr,Garland,Tx,,,,,,
62947,,,,,,,,,


In [14]:
splitTest[1]=splitTest[1].str.replace("LKB|TK L|Th Floor|Suite|^#|^#a|tria|Water St","")
splitTest[1]=splitTest[1].str.replace("#|\(C\/O Wework\)|& Concord Pike|% Corp. Tax Dept|C\/O.*.","")
splitTest[1]=splitTest[1].str.replace("Inc\.|Inc|Ste\.$|Ste$|Ste $|Unit [A-Z]|.*.Blvd\.|.*.Blvd","")
splitTest[1]=splitTest[1].str.replace("W Th St|Ste\. [A-Z]|Ste [A-Z]|.*.Drive|Townpark Dr\.N\.W\.","")
splitTest[1]=splitTest[1].str.replace("eastern Tax Region P\.O\. Box|Halstead St|Duke Of Gloucester St|Two Pitcairn Place","")
splitTest[1]=splitTest[1].str.replace(".*.Street|Innovation Dr|.*.Pkwy|Rockledge Dr|Ste [A-Z][a-z]|Kenilworth Dr","")
splitTest[1]=splitTest[1].str.replace("StePmb|Attn: Carole I\. Amster|Steinway Pl|R\. Calvert Steuart  Basil Court","")
splitTest[1]=splitTest[1].str.replace("Stephen R Glaser Md|\.|P\.O\. Box|Building [A-Z]|Shermantowne Dr.","")
splitTest[1]=splitTest[1].str.replace("Box|.*.Road|S\. Gayoso St\.|Llc|Po Box| L\.L\.C\.  Beards Dock Crossing","")
splitTest[1]=splitTest[1].str.replace("Box  [A-Z]|Crestwood Parkway Nw|.*.Avenue|.*.Center|Cemetery Rd","")
splitTest[1]=splitTest[1].str.replace("P O Box [A-Z]|Th Flr|P O Box|.*.Floor|Tax Dept.*.|Nd|White Bear Twp","")
splitTest[1]=splitTest[1].str.replace("The Woodlands|N\.E\.|Old Country Rd|Rm|Yale Road|Pmb|Tormey Lane Ne|Apt\.|Apt","")
splitTest[1]=splitTest[1].str.replace("Apt\. [A-Z]|Apt [A-Z]|Unit|Tax Bldg [A-Z]|S\.|.*.Hwy|Mi  Unit|Clinton Township","")
splitTest[1]=splitTest[1].str.replace("Attn\:.*.| Esplanade Way| Dr|Exchange Place|D ApD|The Villages|Th Fl|.*.Ave\.$","")
splitTest[1]=splitTest[1].str.replace(".*.Ave\.$|Ste $","")
splitTest[1]=splitTest[1].str.replace("Jr\.|Route W North|.*.Lane|The Woodlands|.*.Highway|E Broad St|Office Bldg|West Th St","")
splitTest[1]=splitTest[1].str.replace("East Th St|Cronhill Rd|Box [A-Z]|I Javier E. Zavaleta  Applitree La","")
splitTest[1]=splitTest[1].str.replace("Lottes Dr|Jam Packaging Llc Dba Ae Global|A Cooperative|","")
splitTest[1]=splitTest[1].str.replace("Columbia Gateway Dr|John Ross Court|Walker St|Bldg\.|Bldg|Attn.*.|Ltd\.|Ltd","")
splitTest[1]=splitTest[1].str.replace("Bank St|Windsor St|rease Miller Rd|Building|N\. Florida Mango Rd\.","")
splitTest[1]=splitTest[1].str.replace("Wwashington St\.|Box  \(Woodland Dr\.\)|Swan Lake Rd\.|Anne Ziebell","")
splitTest[1]=splitTest[1].str.replace("Legal Department|N Club Dr |S Gayoso St|LLC  Beards Dock Crossing|P O  A","")
splitTest[1]=splitTest[1].str.replace("Colonial Rd|P O|HqE|Post Office|Ft Smallwood RdPegasus Technology","")
splitTest[1]=splitTest[1].str.replace("One Seagate|North Main St|N MainBoston Pst Rd|Penn Square East|Z C","")
splitTest[1]=splitTest[1].str.replace("I Javier E Zavaleta  Applitree La|Jam Packaging  Dba Ae Global|\(Woodland\)","")
splitTest[1]=splitTest[1].str.replace("Swan Lake Rd|Larimer St| Bld [A-Z]|LLC|Truck House Rd|N Woodyear St","")
splitTest[1]=splitTest[1].str.replace("Box|tin$|Ne  C|M\/S Sc|A Brazos St|N Th St|Cornell University|W  St","")
splitTest[1]=splitTest[1].str.replace("Eli Lilly And Co State Tax Group|A Division Of Ptt|Floor|\/|E State St","")
splitTest[1]=splitTest[1].str.replace("One Landmark Square|Mile Rd| Of The Stars|W Trade St|Vincent Rd","")
splitTest[1]=splitTest[1].str.replace("Massachusetts General Hospital East|One Constitution Plaza|Langdrum Ln","")
splitTest[1]=splitTest[1].str.replace("Dunwoody Rd|West Bloomfield Township|POawer|J J Ghingher|York And Warren Rds","")
splitTest[1]=splitTest[1].str.replace("Po Bx|So Long St| Accounting Manager|Accounting Manager|W Tuscarawas St|E  St","")
splitTest[1]=splitTest[1].str.replace("orporated| Mountain Rd|West Bloomfield|Ruxton Rd|reet|Joseph Newgarden Iii","")
splitTest[1]=splitTest[1].str.replace("Broadhollow Rd| Mountlake Ter|South Tower|Elm St|Conklin St|Nw","")
splitTest[1]=splitTest[1].str.replace("South Third St West|Lincoln Ave|Wentling Ave|Washington Ave S|Office ","")
splitTest[1]=splitTest[1].str.replace("S Columbus Ave|West Fairbanks Ave|Jr|Ecm|Port Rd|Park Ave|University Place","")
splitTest[1]=splitTest[1].str.replace("N Florida Mango Rd| Esq|Wwashington St|Day Ave|Wabash Ave|Washington Ave","")
splitTest[1]=splitTest[1].str.replace("Lewis Ave|W Grand Ave|Ph$|Seventh Ave|Park Ave S|South St|SW Th Ave","")
splitTest[1]=splitTest[1].str.replace("Airport|Lockman Ave|Shelby Township  Madison Ave|Theodore Fremd Ave","")
splitTest[1]=splitTest[1].str.replace("N Front St Fl RSt|W Grand River Ave|Crane St|Merchant St Fl |Third Ave","")
splitTest[1]=splitTest[1].str.replace("Shelby Township|Madison Ave|Rothschild|Number|Ste$|Flexitoys|^os$|  Ave$","")
splitTest[1]=splitTest[1].str.replace(" Ashwood Parkway|^Cs$|^Tus$| Boston Pst Rd|Ext$|Esplanade Way|Center|Benton","")
splitTest[1]=splitTest[1].str.replace("Fuquay Varina| Ne$|David Petschulat|Bld.+[A-Z]|^Na$|Tus Ca |Abe Pollin","")
splitTest[1]=splitTest[1].str.replace("^Rt $| Eastgate Mall|Tax Department|Center|Accoung Manager|Rolling Hills Estate","")
splitTest[1]=splitTest[1].str.replace("Ste[A-Z]$|Ste  W|Ste  $|Ste   $|Hillcrest Campus|NE|Healthtrio|Hassan Almoazen","")
splitTest[1]=splitTest[1].str.replace("^Iv$|^Ne$|^Cr $|^ville$|Buena Park|Loren Skeist|Meridian Centre|Main St|Oliver Tower","")
splitTest[1]=splitTest[1].str.replace("Columbia Gateway|Stamford Plaza|Esq|Ft Smallwood Rd|Echo Hill|Nine|N Crescent","")

splitTest[1]=splitTest[1].str.replace("C Roseville","Roseville")
splitTest[1]=splitTest[1].str.replace("A Fayetteville|Fayetteville","")
splitTest[1]=splitTest[1].str.replace("B San Carlos","San Carlos")
splitTest[1]=splitTest[1].str.replace("Sn Clara","Santa Clara")
splitTest[1]=splitTest[1].str.replace("B Manhattan","Manhattan")
splitTest[1]=splitTest[1].str.replace("Mc Connellsburg"," McConnellsburg")
splitTest[1]=splitTest[1].str.replace("Floor New York","New York")
splitTest[1]=splitTest[1].str.replace("Woodland Rd \(Rear\) Pasadena Md","Pasadena Md")
splitTest[1]=splitTest[1].str.replace("Oriental St Rockville Md","Rockville Md")
splitTest[1]=splitTest[1].str.replace("Lobby A Los Angeles","Los Angeles")
splitTest[1]=splitTest[1].str.replace("C San Diego Ca","San Diego Ca")
splitTest[1]=splitTest[1].str.replace("A San Marcos Ca ","San Marcos Ca ")
splitTest[1]=splitTest[1].str.replace("Mpls","Minneapolis")
splitTest[1]=splitTest[1].str.replace("Beverly Hl","Beverly Hills")
splitTest[1]=splitTest[1].str.replace("Miami           Florida|Miami           Florida","Miami Florida")
splitTest[1]=splitTest[1].str.replace("Ste  St Louis|Ste  Saint Louis","St Louis")
splitTest[1]=splitTest[1].str.replace("Ste  Los Angeles Ca ","Los Angeles Ca ")
splitTest[1]=splitTest[1].str.replace("Santa Rsa Bch","Santa Rosa Beach")
splitTest[1]=splitTest[1].str.replace("Ste  Oakland Ca ","Oakland Ca")
splitTest[1]=splitTest[1].str.replace("   Ste  Bethesda Md ","Bethesda Md")
splitTest[1]=splitTest[1].str.replace("Ste  Valley Village Ca ","Valley Village Ca")
splitTest[1]=splitTest[1].str.replace("   Ste  Towson Md ","Towson Md")
splitTest[1]=splitTest[1].str.replace("Ste  Manhattan Beach Ca ","Manhattan Beach Ca")
splitTest[1]=splitTest[1].str.replace("Ste  Baltimore Md ","Baltimore Md")
splitTest[1]=splitTest[1].str.replace("Ste  Little Rock","Little Rock")

In [15]:
splitTest[0]=splitTest[0].str.replace("LKB|TK L|Th Floor|Suite|^#|^#a|tria|Water St","")
splitTest[0]=splitTest[0].str.replace("#|\/ [A-Z].+|\. Rd Flr|[A-Z].+Boulevard|[A-Z].+Blvd|P\.O\. Box","")
splitTest[0]=splitTest[0].str.replace("Th Street|N Quaker Rd\.|Cloud Dx|Carondelet.+Ave.*|Ninth Street Se\.","")
splitTest[0]=splitTest[0].str.replace("[W,N,S,E] .+Ste|[A-Z].+Road|[W,N,S,E].+Street|[A-Z].+Way|Nw.+Place|Nw.+Ave|Nw.+Dr","")
splitTest[0]=splitTest[0].str.replace("[W,N,S,E].+Avenue|[W,N,S,E].+St|Post.+Box|Po.+Box|P O.+Box|[A-Z].+Inc|[A-Z].+Inc\.","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Circle|Ne$|[A-Z].+St|[A-Z].+Drive|[A-Z].+Rd|[A-Z].+Corp|[A-Z].+Court","")
splitTest[0]=splitTest[0].str.replace("C\/O.+|Po Bx|[A-Z].+Ct|[A-Z].+Avenue|[A-Z].+Floor|[A-Z].+Trace|POBox","")
splitTest[0]=splitTest[0].str.replace("e A$|Ste [A-Z]|[A-Z].+Ave|[A-Z].+Lane|[A-Z].+Path|[A-Z].+Place|Th.+Ave","")
splitTest[0]=splitTest[0].str.replace("tralian.+South|[A-Z].+Ln|[A-Z].+Technology|[A-Z].+%|%.+[A-Z]|\(.+\)|;.+","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Terrace|[A-Z].+Parkway|.+Company|[A-Z].+Pkwy|[A-Z].+Dr|[A-Z].+Hwy|Unit.+","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Trl|[A-Z].+Ave$|.*.Ave$|.*.Ave $|[A-Z].+Solutions|RouteBox.+","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Ave$|.*.Ave$|.*.Ave $|[A-Z].+Sales|[A-Z].+Cv|[A-Z].+Centre|[A-Z].+Highway","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Technologies|[A-Z].+Laboratories|Ste $| Ste $|[A-Z].+Materials|P O.+Box ","")

splitTest[0]=splitTest[0].str.replace("Th St. South|Palm Coast Pkwy Ne|Twig Ter|\.|Brewton Cr|Th Terr North|Avenue","")
splitTest[0]=splitTest[0].str.replace("reet| N Pastoria Ave| tin Street| Seminary Dr E|Cambridge Ave|Gayley|Fairway","")
splitTest[0]=splitTest[0].str.replace("  Rnb| Alamo Wy| Joseph Ct|Louis Russo| Gullwing| Middlebrook Dr| S Lemon Ave  ","")
splitTest[0]=splitTest[0].str.replace(" La Mirada| St Ave South| West$| Hanna Woods| Cleghorn Ave| Lovers Ln| tralian Ave e ","")
splitTest[0]=splitTest[0].str.replace("Jacques Moret| Penn Plaza | San Elijo Ave | Erie Dr| e C| e A| N Tyler Ave| Th Ave ","")
splitTest[0]=splitTest[0].str.replace(" N Arlington Ave|\)|\(Eloise| Aukai Ave| Huntwood Ave|  Central| Ne Granger Ave","")
splitTest[0]=splitTest[0].str.replace(" So Market Bldg Faneuil Hall Sq|None Listed|Box  [A-Z] | Century Park e |","")
splitTest[0]=splitTest[0].str.replace(" Katella Ave | E Fowler Ave| Conroye |Frank V Vitale| Avenida Encinas  C| Broadway ","")
splitTest[0]=splitTest[0].str.replace(" Camino Del Rio N  | Ne Th Pl|Anthony Mazzeo| Fieldcrest Dr| HuffsmithConroe","")
splitTest[0]=splitTest[0].str.replace(" Brumley Ave| Brazos e |Baristo| Ne  | Alexander Ave| Ford| S Raymond Ave","")
splitTest[0]=splitTest[0].str.replace(" Bridgeport Ave| Quebrada Cir|  Byers Hall  Mc  |Raxco Software| Ne Apt ","")
splitTest[0]=splitTest[0].str.replace("Cryomedical Sciences| Baldwin Pl| El Camino Real| Latchford Ave|oration|  Flr","")
splitTest[0]=splitTest[0].str.replace("The Brenn Er Group Inc|  Calvert| Century Park E| S Kelton Ave|A Pmb | E El Camino Real ","")
splitTest[0]=splitTest[0].str.replace("  Th Fl| Great Plain Ave |W Frear Innovations|One Canal Park|RQC Ltd| E  Bloomingham Ave","")
splitTest[0]=splitTest[0].str.replace(" S Beverly Glen |Cri Ventures| E Lincoln Ave|Halo Sports And Safety| Th Ter N","")
splitTest[0]=splitTest[0].str.replace(" North Glendora Ave| Vail Ave| Forbes Ave|Randomperfect| J Kent Adams| Nueces","")
splitTest[0]=splitTest[0].str.replace(" Robert L Barton| Commscope Pl Se|Manish Damani|Old|Pmb| S Eastern Ave| Railroad Pl","")
splitTest[0]=splitTest[0].str.replace("Rt| Air Brake Ave| S Atchison Cir|Apis Cor Engineering| Tomas| Avenue Of e |Thomas Kerl","")
splitTest[0]=splitTest[0].str.replace(" Preston Royal e | E Country View Cir| Cornhusker| Camino Vida Roble | Prospect Row","")
splitTest[0]=splitTest[0].str.replace(" Mt Pleasant S| West Cummings Pk|Fernway|UkAstecc| Montebello R |B Arroyo Cuyamungue","")
splitTest[0]=splitTest[0].str.replace(" Entrada Empinada| Virginia Se| S House A| Via Colinas| Calle Rivera| S Claybrook","")
splitTest[0]=splitTest[0].str.replace(" ars|David Teller| Bryan e |Naresh Wadhwa|George Kaladjian|  Ext$| South Broadway","")
splitTest[0]=splitTest[0].str.replace(" Pacific e | Vonkarman | Poplar Isle| Ampole| N Military Trail| Rider Trail S","")
splitTest[0]=splitTest[0].str.replace(" N Camino Padre Isidoro| N Custer Ave| Cotting Ave|Connectyourcare|Llc|Llc\.","")
splitTest[0]=splitTest[0].str.replace("   Apt $| S Hous$|POBox  |  Dr$| arr Rd$|  Rm $| Emerald$| Jinetes$|^ Rd $|  Of e $","")
splitTest[0]=splitTest[0].str.replace(" Woodland Av|  Rm [A-Z]$| Evereve Cir$| Wilma Rudolph B| Hamilton Natl B|Apt $","")
splitTest[0]=splitTest[0].str.replace("Th Fl Blue Cross Bd| [A-Z].+Av$|ephen Benton| L B Mcleod| [A-Z].+Plaza |P O Box $","")
splitTest[0]=splitTest[0].str.replace(" Avenida Acaso|PemAmerica|Signtex| N Bowling Green Wy| Clairemont Mesa Bl","")
splitTest[0]=splitTest[0].str.replace("John Pufahl|   Ff$|One Post Office Square$| Donald Douglas Loop South|TheriTeck","")
splitTest[0]=splitTest[0].str.replace(" [A-Z].+Commerce$| Argonaut| New England Executive Park|ast $|^ Cr $| West Cummings Park","")
splitTest[0]=splitTest[0].str.replace(" Camino Del Rio South| Park Plaza| e D$|Buffalo Automation Group| Tartan Pl|^ nue$","")
splitTest[0]=splitTest[0].str.replace("  Terrace|^ e G$|Indusl Rowe| e  Lb $|William Robertson  | Market  Tower","")
splitTest[0]=splitTest[0].str.replace("^ SE $| Southeast  Causeway|Terrance Herron| SW  Terr|Lawrence P Dimarzio","")
splitTest[0]=splitTest[0].str.replace(" Investment Partners| E Via Sabia| Progress Cir| e B$| N Gemini Pl| Nw  Sy","")
splitTest[0]=splitTest[0].str.replace(" [A-Z].+Trail|PolyPak Industries| Villa e | W Jackson| Commerce e |  Bldg $","")
splitTest[0]=splitTest[0].str.replace(" Alhambra Cir| [A-Z].+Square| ive$| Honeysuckle Cir|PO  Box |","")
splitTest[0]=splitTest[0].str.replace("[A-Za-z].+Management|[A-Za-z].+Services|[A-Za-z].+Oncology|James River Paper Co","")
splitTest[0]=splitTest[0].str.replace("   eir$| Stanbery Ridge|Borderfree| Castro  $|Csx Transportation| Hawthorne Bl","")
splitTest[0]=splitTest[0].str.replace(" Nottingham Pl| Eastgate Mall H| Washington$|Taggents|Four Liberty Square| Calle Patricia","")
splitTest[0]=splitTest[0].str.replace("P O Bx  Tax Dept|Clarifai|Jeffrey Pulver|PulverCom| Bayfront Plaza|KanoodleCom","")
splitTest[0]=splitTest[0].str.replace(" North Belvedere| Calle Amanecer|[A-Z].+Laboratory|US Macrogen| Sterling Ridge","")
splitTest[0]=splitTest[0].str.replace(" Hamilton Rm | Camino Molinero| e E & F| Mariner Square Loop| Spectrum","")
splitTest[0]=splitTest[0].str.replace(" Franklin e | Rue Bordeaux| Robert Bril| Congress e | e  Box | N Fr | M Center","")
splitTest[0]=splitTest[0].str.replace("Minibar North America| E Mesquite| Gates| Highway  S| AA North| Addison e ","")
splitTest[0]=splitTest[0].str.replace("Dynamic Hive|Caraustar Custom Packaging Group \(Maryla| West Foothill A","")
splitTest[0]=splitTest[0].str.replace("Wemtec| Lockhill Selma|Dynoraxx| Froad| Parkway$| Road $|Advanced Reactor Concepts","")
splitTest[0]=splitTest[0].str.replace(" Kenmore| S Garnsey|  Penthouse| North Belmont| Arroyo Cuyamungue| San Expwy","")
splitTest[0]=splitTest[0].str.replace("Attn: Tax.+|Jack W Silberlicht|A Kings Row| Marwood| Camino Leonor|  Rd Fl$","")
splitTest[0]=splitTest[0].str.replace(" e H$| Cheshire Cr| Century Pk East Rm | MB Martinez| Avenida De La Playa|  Canton$","")
splitTest[0]=splitTest[0].str.replace(" Nw Th Ter| Piedra Morada|Edvotek| udio $| Directors Row|Synacor|[A-Z].+Industries","")
splitTest[0]=splitTest[0].str.replace(" Travis Cir N| Marbella Vista|Irving P Macauley Jr|  Frnt Frnt|Aitheras|[A-Z].+Group","")
splitTest[0]=splitTest[0].str.replace(" Rd e $| Cottage e | S Vinewood|A Ronda Mendoza| Morgan$| Rosehaven e ","")
splitTest[0]=splitTest[0].str.replace(" Arroyo Vista|Manhattan Bridge Capital| Meadowbrook Wy|Jefrey Pulver| Loop|Guidecraft","")
splitTest[0]=splitTest[0].str.replace(" High Meadow|Box $|Box [A-Z]$| Broadway| E Merced|Duane A Ball|  orzewski","")
splitTest[0]=splitTest[0].str.replace(" emmons Frwy| E Century Park |Gridkicker| [A-Za-z].+Bldg| [A-Z].+Country Club","")
splitTest[0]=splitTest[0].str.replace(" tin e |[A-Za-z].+Bank|[A-Z].+Distribution| Boston Scientific Pl|Attn: Legal|  Hwy \/","")
splitTest[0]=splitTest[0].str.replace(" Puerta Del Sol| Monroe Wy| E Camino Pinzon| S Mt Vernon| Orange e F| Timbergreen Cir","")
splitTest[0]=splitTest[0].str.replace("[A-Za-z].+Fund| Market Pl | Hwy $|Saame|Nd Flr|[A-Za-z].+Llp","")
splitTest[0]=splitTest[0].str.replace("orate Place|E Lake Cir|N  W|Jacobs Run|E El Cmaino Real|Palailai Pl|nida Santiago","")
splitTest[0]=splitTest[0].str.replace("Gloria M Buley|East Coast Dyes|N Lemon|E Olympic|Tax Dept |Via Rivera|Green Peak Orchard","")
splitTest[0]=splitTest[0].str.replace("Th Pl|[A-Z].+Entertainment|Route$|Cricklewood|Keyme|Camino Ramon|J A Webb","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Associates|Rxr Plaza|W Alameda|Vrcaffeine|Neuro Probe|William Henry|Cuyamaca e","")
splitTest[0]=splitTest[0].str.replace("Singer|Kendall Sq|Teledyne|Apt C|Universal City Plaza|Calle Juno|Orthogonal","")
splitTest[0]=splitTest[0].str.replace("East Camino Real|Southwest|Nicolas Fromanger|Oncoimmunin|[A-Za-z].+Systems|Dalemead","")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Plaza","")

splitTest[0]=splitTest[0].str.replace(" e  San Jose Ca ","San Jose Ca ")
splitTest[0]=splitTest[0].str.replace("  Ste  San Mateo Ca | e  San Mateo Ca ","San Mateo Ca")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Los Angeles Ca ","Los Angeles Ca")
splitTest[0]=splitTest[0].str.replace("[A-Z].+Fargo","Fargo")
splitTest[0]=splitTest[0].str.replace(" Road  Tulare Ca ","Tulare Ca")
splitTest[0]=splitTest[0].str.replace("  Sterling Hts Mi ","Sterling Heights Mi")
splitTest[0]=splitTest[0].str.replace(" e  Menlo Park Ca ","Menlo Park Ca")
splitTest[0]=splitTest[0].str.replace("  N Twin Falls| Ave W Twin Falls| S Twin Falls","Twin Falls")
splitTest[0]=splitTest[0].str.replace("  East Palo Alto Ca |   A Palo Alto Ca ","Palo Alto Ca")
splitTest[0]=splitTest[0].str.replace("  Bldg  Coraopolis","Coraopolis")
splitTest[0]=splitTest[0].str.replace(" nt Dr Oxford","Oxford")
splitTest[0]=splitTest[0].str.replace(" NSanta Ana Ca ","Santa Ana Ca")
splitTest[0]=splitTest[0].str.replace(" Endeavor Cir Brea Ca ","Brea Ca")
splitTest[0]=splitTest[0].str.replace("A E Battlefield   Springfield","Springfield")
splitTest[0]=splitTest[0].str.replace(" e  Falls Church","Falls Church")
splitTest[0]=splitTest[0].str.replace("E Compton Ca","Compton Ca")
splitTest[0]=splitTest[0].str.replace("Ste  Cupertino Ca","Cupertino Ca")
splitTest[0]=splitTest[0].str.replace("P O Box  Orofino","Orofino")
splitTest[0]=splitTest[0].str.replace("Box  Buhl","Buhl")
splitTest[0]=splitTest[0].str.replace("S Federal Wy Boise","Boise")
splitTest[0]=splitTest[0].str.replace("S Th W Apt  Rexburg","Rexburg")
splitTest[0]=splitTest[0].str.replace("e  F Beverly Ma","Beverly Ma")
splitTest[0]=splitTest[0].str.replace("e  Huntsville","Huntsville")
splitTest[0]=splitTest[0].str.replace("B Ojai Ca","Ojai Ca")
splitTest[0]=splitTest[0].str.replace("Idaho Avnue Santa Monica Ca","Santa Monica Ca")
splitTest[0]=splitTest[0].str.replace("e  Columbia Md","Columbia Md")
splitTest[0]=splitTest[0].str.replace("Mission  Vicksburg","Vicksburg")
splitTest[0]=splitTest[0].str.replace("Box  Girard Oh","Girard Oh")
splitTest[0]=splitTest[0].str.replace("e  Berkeley Ca","Berkeley Ca")
splitTest[0]=splitTest[0].str.replace("A Via Rancho San Diego","San Diego")
splitTest[0]=splitTest[0].str.replace("Tamarack Twin Falls","Twin Falls")
splitTest[0]=splitTest[0].str.replace("W  S Aberdeen","Aberdeen")
splitTest[0]=splitTest[0].str.replace("S Payette","Payette")
splitTest[0]=splitTest[0].str.replace("Th And s Frederick Md","Frederick Md")
splitTest[0]=splitTest[0].str.replace("Ste  Santa Monica","Santa Monica")
splitTest[0]=splitTest[0].str.replace("South  Rogers","Rogers")
splitTest[0]=splitTest[0].str.replace("Roger B Chaffee Se Grand Rapids Mi","Grand Rapids Mi")

In [16]:
splitTest[0]=splitTest[0].str.lstrip()
splitTest[0]=splitTest[0].str.rstrip()

splitTest[1]=splitTest[1].str.lstrip()
splitTest[1]=splitTest[1].str.rstrip()

splitTest[2]=splitTest[2].str.lstrip()
splitTest[2]=splitTest[2].str.rstrip()

splitTest[3]=splitTest[3].str.lstrip()
splitTest[3]=splitTest[3].str.rstrip()

splitTest[4]=splitTest[4].str.lstrip()
splitTest[4]=splitTest[4].str.rstrip()

splitTest[5]=splitTest[5].str.lstrip()
splitTest[5]=splitTest[5].str.rstrip()

splitTest[6]=splitTest[6].str.lstrip()
splitTest[6]=splitTest[6].str.rstrip()

d=len(splitTest)
nList=[]

for i in range(d):

    if 1 <= len(splitTest[0][i]) <= 4:
        nList.append("")
        
    else:
        nList.append(splitTest[0][i])

In [17]:
splitTest['nCol']=nList

splitTest['nCol']=splitTest['nCol'].str.replace("^South$|^North$|^East$|^West$|Route$|Drive$|Attn: Tax|Via Del Oro|East Main|","")
splitTest['nCol']=splitTest['nCol'].str.replace("Gothard e I|Chimney Wood|Tesla Capital|Via Pasar|J A Andrews|W Dade|One e","")
splitTest['nCol']=splitTest['nCol'].str.replace("Metro Center|Unknown|Th Flr|Bosworth|tin Road$|Venman & Co|Newtown Pike","")
splitTest['nCol']=splitTest['nCol'].str.replace("Highland Glen Pl|PO Box\:|Montague Expressway|ate Rd|Population Bio Uk","")
splitTest['nCol']=splitTest['nCol'].str.replace("Providence Pike|Fairmount e|Custom Optical Frames|Hwy M Imperial|Chrysler","")
splitTest['nCol']=splitTest['nCol'].str.replace("Bldg [A-Z]|Cordova|Hwy  South|W Cesar Chavez|Wise Electronics|Freedom Cir|Calle Plano","")
splitTest['nCol']=splitTest['nCol'].str.replace("Bungalow|E F Yarbrough|North Fwy|Road H|Paul J Weller|San Francisco Ter|Morning Glory Cir","")
splitTest['nCol']=splitTest['nCol'].str.replace("Third e|Knights Bridge Pl|Jenner|Infinite|Highwaye|Oncormed|Principal Row","")
splitTest['nCol']=splitTest['nCol'].str.replace("Cem Machine|Bishop e|Camino Alaegrae|Theodore Cha|Hatcher Creek|State $|[A-Z].+Products","")
splitTest['nCol']=splitTest['nCol'].str.replace("Ameriprise Financial|Buzztable|W Olympic Bl|Hwy  [A-Z]|Tax Dept|Frostwood","")
splitTest['nCol']=splitTest['nCol'].str.replace("ar Dr|Ave E|N Western|W Magnolia|Newport Cir|Marina Vista|Sand Canyon|Kassarine Pass","")
splitTest['nCol']=splitTest['nCol'].str.replace("Camino Del Vistazo|Spring Cor|Vaughan Mall|The Power House.*|Lakeside$","")
splitTest['nCol']=splitTest['nCol'].str.replace("Rullion Greenway|Avram A Glazer|Nd Fl|Larwin Cir|Mount Auburn e|Of N Caro","")
splitTest['nCol']=splitTest['nCol'].str.replace("E Montebello|Calle Alameda|Rain King Software|Scott Verost|Reade Pl|W Alvarez","")
splitTest['nCol']=splitTest['nCol'].str.replace("Peabody Indusl Center|Elmscourt Cir|Summit Ridge|Spring Violet Pl|Pier  e","")
splitTest['nCol']=splitTest['nCol'].str.replace("Montgomery e|H Henry Brennan|Trowbridge|At Th Terr|Virginia e|Via De La Amistad","")
splitTest['nCol']=splitTest['nCol'].str.replace("South Fwy|[A-Z].+Center|Peachtree|Robert E Batson|Macarthur Bl East Tower|Apt [A-Z]","")
splitTest['nCol']=splitTest['nCol'].str.replace("Townsend e|Springboro Pike|Highway|Ballardvale|Park Sorrento|Fargo Capital Center","")
splitTest['nCol']=splitTest['nCol'].str.replace("Via Real|Q Link Wireless|Mt Baldy Cir|High Brush Cir|S Hawthorne Hollow Cir","")
splitTest['nCol']=splitTest['nCol'].str.replace("e  oore|Lee Pl|Albany Cir|Breeze Hill|N  Cir|Advanced Power Devices|San Simeon","")
splitTest['nCol']=splitTest['nCol'].str.replace("Oak Ridge Turnpi|Ne Hurley N|S Camino Porvenir|W Pierce Butler|Tesconi Cir","")
splitTest['nCol']=splitTest['nCol'].str.replace("Paseo Del Sol|Flying Cloud Course|Factual|Kroll Factual Data|KG Motors|Cheri Cir","")
splitTest['nCol']=splitTest['nCol'].str.replace("Renard Pl e|La Paloma|U S  South|C N Capital Of Tx|Saticoy|Middlesex Turnpike","")
splitTest['nCol']=splitTest['nCol'].str.replace("Linwell|Second|Johnston Bldg|Village Green|Post Office Sq|Technology Sq","")
splitTest['nCol']=splitTest['nCol'].str.replace("Place|Woodpecker Ridge|Baltimore Biomedical|Wt Montgomery|ar Rd|Via Regla","")
splitTest['nCol']=splitTest['nCol'].str.replace("Global Media Network|[A-Z].+Enterprises|Astor M ValHackett|Michael T Mehta|Regal Row","")
splitTest['nCol']=splitTest['nCol'].str.replace("Jarrod M Espostio|N Wallace|B Four Seasons|ortion|Amber Gln|OuterSeal Of Metropolitan","")
splitTest['nCol']=splitTest['nCol'].str.replace("Prestwick Cir|Sapling Spg|Post Office Sq|BuildingWest|Multispec Research & Development Div","")
splitTest['nCol']=splitTest['nCol'].str.replace("Secondwrite|[A-Z].+Holdings|Nagog Park|Agnik|E Fire Agate Pl|George F Hammel","")
splitTest['nCol']=splitTest['nCol'].str.replace("Mauchly|One Mauchly|Branding Iron|Whisker Labs|Innerbelt Business Center|E Harmony","")
splitTest['nCol']=splitTest['nCol'].str.replace("Telkonet.*|Marie Hayes|Salta Verde Point|Sonavi Labs|Del Mar Ca|Of The Americas","")
splitTest['nCol']=splitTest['nCol'].str.replace("Moose Tr|Appaloosa|Devils Lake|Apt H|Hekaha e|Alakea e|Conrae|Level|Avenida Consentido","")
splitTest['nCol']=splitTest['nCol'].str.replace("Ne Tillamook|Roosevelt|Hoover Containment|W Hacienda|ast Rm|Via Azalea","")
splitTest['nCol']=splitTest['nCol'].str.replace("E Greenway Plz|MidContinent Tower|Cardinal Pl|Fairbanks|Harpoon Medical|Oxnard e","")
splitTest['nCol']=splitTest['nCol'].str.replace("Maleberry Run|  Dpt| One$|Monarch Bay|Pontius|Rialto Pl|Derby Downs","")
splitTest['nCol']=splitTest['nCol'].str.replace("ast Rm|Via Azalea|S Los Robles|St North|West Columbia|The Skating Club Of Florida","")
splitTest['nCol']=splitTest['nCol'].str.replace("Fourth Flr|Halcyon|West University|Gms Mine Repair And Maintenance|Farrington Pl","")
splitTest['nCol']=splitTest['nCol'].str.replace("Cypress|College Park|Alfred Cir|Douglas M Mci|allard Pl|Sw Excalibur Pl","")
splitTest['nCol']=splitTest['nCol'].str.replace("Central Expressway|Platino|Vanderbilt|Herbal|Merryfield Row|Beacon Capital Partners","")
splitTest['nCol']=splitTest['nCol'].str.replace("Technology Pl|B Cynthia Nw|N Ninth|Building|S Rustic Cir|Minto|Morgan C|West South","")
splitTest['nCol']=splitTest['nCol'].str.replace("Ronway Fashion|Administration eacham Field|arner|Sterling Cir|Sw Creekside|Dupont|N Van Buren","")
splitTest['nCol']=splitTest['nCol'].str.replace("Silverfox Dental & Ortho|Federal e|ar Hwy|Convoy Crt|Corsearch|Ronald Bradford","")
splitTest['nCol']=splitTest['nCol'].str.replace("Paradise Wy|Westerly Pl|Roseheart|Advanced Decision Handling|Am Cntr|E Sunnyoaks|Brookhurst","")
splitTest['nCol']=splitTest['nCol'].str.replace("Hansen|Nw Flanders|Wilma|Bozeman|Les Horvath|Avenu|Hulen e|Quitman|Embassy Unit","")
splitTest['nCol']=splitTest['nCol'].str.replace("S Taylor|einer|NDenver|Bishop Cir|ney Campbell Causeway|Oregon Pike|iq|Synphagen","")
splitTest['nCol']=splitTest['nCol'].str.replace("Portland Av S|Cascade Maverik Lacrosse|Henrietta Creek|Dominica Cir|International","")
splitTest['nCol']=splitTest['nCol'].str.replace("Evergreen Cir|Myrtle Crk|Reality Analytics| Wy$|Cudahy Pl|S Ellsworth|Mercury Point","")
splitTest['nCol']=splitTest['nCol'].str.replace("Merritt|Strawberry Hill|Main e|Court|Bissonnet e|Park Pl|E Alpaca|Sea Lion Pl","")
splitTest['nCol']=splitTest['nCol'].str.replace("West  South|Aging Innovations|atson|Lawrence W Matson|Waterview Cir|Presidential Gtwy","")
splitTest['nCol']=splitTest['nCol'].str.replace("ennington|Sharycircle|Floor|Bioqual|Brannan e|Glaser Vision|Network ers Association","")
splitTest['nCol']=splitTest['nCol'].str.replace("S Leigh|Cerro Del Alamo|Country Club|C Nakomis|Phillip M Edewards|uck Specialty Contracting","")
splitTest['nCol']=splitTest['nCol'].str.replace("Edwin L Miller|System|Steinway And Sons|Fannin|Halifax|Columbine e|Ridgeland|East  B","")
splitTest['nCol']=splitTest['nCol'].str.replace("Palmitas|Devon Green|Boyston e|Calle Pintoresco|Via Fabricante|RegentSheffield","")
splitTest['nCol']=splitTest['nCol'].str.replace("Via Amarilla|Element Consulting|Element K|Basic Element|ephenson|Mafatime Fall","")
splitTest['nCol']=splitTest['nCol'].str.replace("Wterworks Wy|Jinete|San Clemente Wy|Louisiana e|North Lbj|Oakleaf Cir|Ave G","")
splitTest['nCol']=splitTest['nCol'].str.replace("Vaxim|Thermolift|De Mille|Moody|eSalem|cgraw|ottingham|S Main|Camino Vida Roble","")
splitTest['nCol']=splitTest['nCol'].str.replace("Coral Cir|Freeman|Fubotv|ation Dr|Upper Miller Creek|MccannErickson Marketing","")
splitTest['nCol']=splitTest['nCol'].str.replace("E California|California e|Jose Manuel Tost|Ste  E|Stuie|Payton|Puerto Del Sol","")
splitTest['nCol']=splitTest['nCol'].str.replace("Camino Vida Roble|E Jackson|Ferndale Pl|Innovo Biopolymers|Angelica Nurseries","")
splitTest['nCol']=splitTest['nCol'].str.replace("Calle Bolero|A Golden Lantern|Michael Plottel|Attn Mark Brinkman|Camino Del Mar","")
splitTest['nCol']=splitTest['nCol'].str.replace("Camino Esmerado|Frenchporte Ip|Pine e|Norwalk Bl|Ave Of The e|E La Palma|Extension","")
splitTest['nCol']=splitTest['nCol'].str.replace("Pine e|Ave Of The e|E La Palma|Extension|B Venture Cove|Southe|West  South","")
splitTest['nCol']=splitTest['nCol'].str.replace("C And N Car Hauling|Services|C & N.+|N & C|C&N Distributors|C&N Percy Properties","")
splitTest['nCol']=splitTest['nCol'].str.replace("N Sycamore|Blue e| Pl|Amper Music|E Crescent|orier|rinkley|Paseo De Las Americas","")
splitTest['nCol']=splitTest['nCol'].str.replace("Goodyear|Cannonero Cir|T Guina|ierks|Bryon Twr|Harvey E White|Surrey Sq","")
splitTest['nCol']=splitTest['nCol'].str.replace("Jack Ledbet|F E Mclain  N Central|Via El Mercado|East Garden Green|Wilshire Bl","")
splitTest['nCol']=splitTest['nCol'].str.replace("Midway|E Florida Pl|J V Jamison|Kean Cir|Danby Of North America|Calle De Madera","")
splitTest['nCol']=splitTest['nCol'].str.replace("Garrsnlockr|e  Pbm|Independence Bl|Willow Grove|e  Pbm|St George Ut|Independence Bl","")
splitTest['nCol']=splitTest['nCol'].str.replace("W Brokaw|Willow Grove|Aloha Blue|R Williamson  S Twr Plz Of The Ameri|Loockerman e L","")
splitTest['nCol']=splitTest['nCol'].str.replace("Psychnostics|Soleo Communications|Seneca Networks|Kingston Pike","")
splitTest['nCol']=splitTest['nCol'].str.replace("Genepeeks|Meadow Hl|Roland Krainz|San Expressway|S Hillcrest","")
splitTest['nCol']=splitTest['nCol'].str.replace("Instantdx|Avery|Rollins Plz|E  North|e   Jamestown|[A-Z].+Cir|[A-Z].+Pike","")
splitTest['nCol']=splitTest['nCol'].str.replace("Giovanni|Heath Hill|Plaza Del Amo|Msa Security|James itt|Freeport","")
splitTest['nCol']=splitTest['nCol'].str.replace("Avelex Limited Partnership|S Croddy|Capital Mercury Apparel|Webscan|First e","")
splitTest['nCol']=splitTest['nCol'].str.replace("Fred E Bassett|Friends|Lakeside Sr|oraiton|San Pedro Se|Four D|Dickinson","")
splitTest['nCol']=splitTest['nCol'].str.replace("Via Valdes|Camino Redondo|San Marino|Mistic Brands|etter|ovett|HighGames","")
splitTest['nCol']=splitTest['nCol'].str.replace("Rue Godbout|Via Del Charro|Timberland|Mission|Kenneth A Zwara|Tyler At Sixth","")
splitTest['nCol']=splitTest['nCol'].str.replace("Intellectual Property Department|Sw Th Terr|Greenwich|Apc Research Assets|So Grant B","")
splitTest['nCol']=splitTest['nCol'].str.replace("Pricewoods|Replubic Natl Bk Bld|Alphapointx|Two NicePak Park|Casa Lindaz","")
splitTest['nCol']=splitTest['nCol'].str.replace("Beechnut e|Airport Fwy|Sienna Biotech|e  Palmz N|A Weill Hall","")
splitTest['nCol']=splitTest['nCol'].str.replace("Alameda De Las Pulgas|Roastingant|Stoneridge Mall|Town And Country Adult Living|Inteleos","")
splitTest['nCol']=splitTest['nCol'].str.replace("E Nettleton|Pacific Privado|e  ernandez|Micro Powders|Micro Properties|Nagog Pk","")
splitTest['nCol']=splitTest['nCol'].str.replace("East  South|Katy Fwy|S Rogers Cir|Blue Ravine|Corporate Park C","")
splitTest['nCol']=splitTest['nCol'].str.replace("Quoizel|Premier Row|Rio Nedo|Harris Specialty Chemicals|S Cedros","")
splitTest['nCol']=splitTest['nCol'].str.replace("Avery|Rollinsz|E  North| C & N|Mainst|write|Arborway|Lower |E Florida","")
splitTest['nCol']=splitTest['nCol'].str.replace("Upper e|R Williamson  S Twrz Of The Ameri|Wterworks|Nobel Biosciences","")
splitTest['nCol']=splitTest['nCol'].str.replace("Woodway|Woofound|C & N","")

splitTest

,0,1,2,3,4,5,6,9,10,nCol
0,,Round Rock,Tx,,,,,,,
1,Huntsville,Al,,,,,,,,Huntsville
2,Alexandria,Al,,,,,,,,Alexandria
3,Vista e,San Diego,Ca,,,,,,,Vista e
4,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
62944,,,,,,,,,,
62945,,Fort Worth,Tx,,,,,,,
62946,,Garland,Tx,,,,,,,
62947,,,,,,,,,,


In [18]:
splitTest1=splitTest.iloc[:,[7,1,2,3,4,5,6]]

splitTest1

,9,1,2,3,4,5,6
0,,Round Rock,Tx,,,,
1,,Al,,,,,
2,,Al,,,,,
3,,San Diego,Ca,,,,
4,,,,,,,
...,...,...,...,...,...,...,...
62944,,,,,,,
62945,,Fort Worth,Tx,,,,
62946,,Garland,Tx,,,,
62947,,,,,,,


In [19]:
splitTest1['nCol']=splitTest1['nCol'].str.lstrip()
splitTest1['nCol']=splitTest1['nCol'].str.rstrip()

splitTest1[1]=splitTest1[1].str.lstrip()
splitTest1[1]=splitTest1[1].str.rstrip()

splitTest1[2]=splitTest1[2].str.lstrip()
splitTest1[2]=splitTest1[2].str.rstrip()

splitTest1[1]=splitTest1[1].str.replace("^South$|^North$|^East$|^West$|Route$|Drive$","")
splitTest1[2]=splitTest1[2].str.replace("^South$|^North$|^East$|^West$|Route$|Drive$","")

e=len(splitTest1)
nList1=[]
nList2=[]
nList3=[]
nList4=[]
nList5=[]
df=pd.DataFrame()

for j in range(e):

    if 1 <= len(splitTest1['nCol'][j]) <= 4:
        nList1.append("")
        
    else:
        nList1.append(splitTest1['nCol'][j])
    
    if len(splitTest1[1][j]) < 2:
        nList2.append("")
        
    else:
        nList2.append(splitTest1[1][j])
    
    if len(splitTest1[2][j]) < 2:
        nList3.append("")
        
    else:
        nList3.append(splitTest1[2][j])
        
    if len(splitTest1[3][j]) < 2:
        nList4.append("")        
        
    else:
        nList4.append(splitTest1[3][j])


splitTest1['n1']=nList1
splitTest1['n2']=nList2
splitTest1['n3']=nList3
splitTest1['n4']=nList4

splitTest2=splitTest1.iloc[:,[7,8,9,10,4,5,6]]

splitTest2['n7'] = splitTest2['n1'] + ' ' + splitTest2['n2'] + ' ' + splitTest2['n3'] + ' ' + splitTest2['n4'] + ' ' + splitTest2[4] + ' ' + splitTest2[5] + ' ' + splitTest2[6]
splitTest3=splitTest2.iloc[:,7:]
splitTest3
display(splitTest3.info(),splitTest3.head())

KeyError: 'nCol'

In [ ]:
splitTest3['n7']=splitTest3['n7'].str.lstrip()
splitTest3['n7']=splitTest3['n7'].str.rstrip()
splitTest3['n7']=splitTest3['n7'].str.lower()

#convert states from full names to two-letter abbreviations
splitTest3.replace({'n7':{"california$":"ca","connecticut$":"ct","massachusetts$":"ma","florida$":"fl","georgia$":"ga",
                          "washington$":"wa","new york$":"ny","delaware$":"de","tennessee$":"tn","missouri$":"mo",
                          "texas$":"tx","indiana$":"in","minnesota$":"mn","pennsylvania$":"pa","oregon$":"or",
                          "virginia$":"va","illinois$":"il","kentucky$":"ky","north carolina$":"nc","new jersey$":"nj",
                          "colorado$":"co","maryland$":"md","ohio$":"oh","arizona$":"az","nevada$":"nv","utah$":"ut",
                          "michigan$":"mi","new hampshire$":"nh","vermont$":"vt","kansas$":"ks","oklahoma$":"ok",
                          "iowa$":"ia","louisiana$":"la","rhode island$":"ri","wisconsin$":"wi","hawaii$":"hi",
                          "montana$":"mt","nebraska$":"ne","district of columbia$":"dc","west virginia$":"wv",
                          "alabama$":"al","idaho$":"id","maine$":"me","new mexico$":"nm","south carolina$":"sc",
                          "north dakota$":"nd","south dakota$":"sd","arkansas$":"ar","alaska$":"ak","wyoming$":"wy",
                          "mississippi$":"ms"}},inplace=True,regex=True)

splitTest4=splitTest3['n7'].str.rsplit(' ',n=1,expand=True)

splitTest4

In [ ]:
splitTest4[1]=np.where(splitTest4[1].isnull(),'missing',splitTest4[1])

nList6=[]

for y in range(len(splitTest4)):
    if len(splitTest4[1][y]) > 2:
        nList6.append("")
    
    elif len(splitTest4[1][y]) == 2:
        nList6.append(splitTest4[1][y])

splitTest4['data_state'] = nList6

splitTest4.replace({'data_state':{"on":"","as":"","vi":""}},inplace=True,regex=True)
splitTest4['data_city']=np.where(splitTest4['data_state'] == '','',splitTest4[0])

splitTest4

In [ ]:
#convert states from full names to two-letter abbreviations
splitTest4.replace({'data_state':{"^al$":"alabama","^ak$":"alaska","^az$":"arizona","^ar$":"arkansas", "^ca$":"california",
                      "^co$":"colorado","^ct$":"connecticut","^dc$":"district of columbia","^de$":"delaware",
                      "^fl$":"florida","^ga$":"georgia","^hi$":"hawaii","^id$":"idaho","^il$":"illinois","^in$":"indiana",
                      "^ia$":"iowa","^ky$":"kentucky","^ks$":"kansas","^la$":"louisiana","^me$":"maine","^ma$":"massachusetts",
                      "^md$":"maryland","^mi$":"michigan","^mn$":"minnesota","^ms$":"mississippi","^mo$":"missouri",
                      "^mt$":"montana","^ne$":"nebraska","^nv$":"nevada","^nh$":"new hampshire","^nj$":"new jersey",
                      "^nm$":"new mexico","^ny$":"new york","^nc$":"north carolina","^nd$":"north dakota","^oh$":"ohio",
                      "^ok$":"oklahoma","^or$":"oregon","^pa$":"pennsylvania","^ri$":"rhode island","^sc$":"south carolina",
                      "^sd$":"south dakota","^tn$":"tennessee","^tx$":"texas","^ut$":"utah","^vt$":"vermont","^va$":"virginia",
                      "^wa$":"washington","^wv$":"west virginia","^wi$":"wisconsin","^wy$":"wyoming","^fla$":"florida",
                      "^arks$":"arkansas"}}
                    ,regex=True,inplace=True)

splitTest5=splitTest4.iloc[:,[3,2]]

splitTest6=pd.concat([df1.iloc[:,:1],splitTest5],axis=1)

display(splitTest6.info(),splitTest6.head())

In [ ]:
splitTest6.replace({'data_state':{"california$":"ca","connecticut$":"ct","massachusetts$":"ma","florida$":"fl","georgia$":"ga",
                          "washington$":"wa","new york$":"ny","delaware$":"de","tennessee$":"tn","missouri$":"mo",
                          "texas$":"tx","indiana$":"in","minnesota$":"mn","pennsylvania$":"pa","oregon$":"or",
                          "virginia$":"va","illinois$":"il","kentucky$":"ky","north carolina$":"nc","new jersey$":"nj",
                          "colorado$":"co","maryland$":"md","ohio$":"oh","arizona$":"az","nevada$":"nv","utah$":"ut",
                          "michigan$":"mi","new hampshire$":"nh","vermont$":"vt","kansas$":"ks","oklahoma$":"ok",
                          "iowa$":"ia","louisiana$":"la","rhode island$":"ri","wisconsin$":"wi","hawaii$":"hi",
                          "montana$":"mt","nebraska$":"ne","district of columbia$":"dc","west virginia$":"wv",
                          "alabama$":"al","idaho$":"id","maine$":"me","new mexico$":"nm","south carolina$":"sc",
                          "north dakota$":"nd","south dakota$":"sd","arkansas$":"ar","alaska$":"ak","wyoming$":"wy",
                          "mississippi$":"ms"}},inplace=True,regex=True)

In [22]:
splitTest6.to_csv("../csvResults/dataFieldCityState_0_65752.csv",index=False)